In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string, re
from autocorrect import spell
from nltk.tokenize.nist import NISTTokenizer
tknzr = NISTTokenizer()
lemmatizer = WordNetLemmatizer()
RE_D = re.compile('\d')
RE_AP = re.compile("'")
RE_DO = re.compile("\.")

import time
from timeout import timeout
import pickle
from math import pow

In [ ]:
def norm(x,n=3):
	y = pow(10,n)
	return int(x*y)/pow(10,n)

p = {}
for i in string.punctuation:
    p[i]=True
def replace(s_out):
    s1_out = []
    for i in s_out:
        try:
            if(p[i]==True):
                continue
        except:
            s1_out.append(i)
    return s1_out

### Read CSV data

In [ ]:
import pandas as pd
df = pd.read_csv('RateMDs.csv')
print(df)

### Adapt off-the-shelf Stopwords

In [ ]:
stop_words = set(stopwords.words("english"))
#print(stop_words)
to_keep = ['she','he','not','during','before','after','him','her','his','hers','herself','himself']
to_remove = ['also',"i'm","i'll","we've","i've","can't","it's","i'd","you're","doesn't","that's"
             'got','would','many','some','few']
for i in to_keep:
    stop_words.remove(i)
for i in to_remove:
    stop_words.add(i)
#print(stop_words)

### Tokenize. Preprocess.

In [ ]:
start = 0
x = 0
for r1 in r[start:]:
#for r1 in r:
    r1 = re.sub('\s+', ' ', r1)
    r_out = tknzr.tokenize(r1)   # or word_tokenize(r1)
    r_out = replace(r_out)
    for i in range(len(r_out)):
        if r_out[i].lower() in stop_words:
            r_out[i] = ''
            continue
        '''if r_out[i][0] > 'A' and r_out[i][0] < 'Z':
            if r_out[i] != spell(r_out[i]):
               print(r_out[i])
        elif r_out[i] != spell(r_out[i]):
            print(r_out[i])'''
        if r_out[i][0] == 'x':
            if spell(str(r_out[i])) == r_out[i]:
                print(r_out[i],spell(r_out[i]))
            else:
                x+=1
print(x)

In [ ]:
import enchant
checker = enchant.Dict("en_US")
to_allow = ['xray','x-ray']
digits = ['0','1','2','3','4','5','6','7','8','9']
@timeout(5)
def pre2(r1): # created June 19
    r1 = r1.lower()
    r1 = re.sub('\s+', ' ', r1)
    r_out = tknzr.tokenize(r1)   # or word_tokenize(r1)
    r_out = replace(r_out)
    for i in range(len(r_out)):
        if len(r_out[i]) < 2:
            r_out[i] = ''
            continue
        if r_out[i][0] in digits:
            r_out[i] = ''
            continue
        if r_out[i] in to_allow:
            continue
        if checker.check(r_out[i]) == False:
            r_out[i] = ''
            continue
        if r_out[i] in stop_words:
            r_out[i] = ''
            continue
        if r_out[i] in names_ml:
            r_out[i] = ''
            continue
        r_out[i] = lemmatizer.lemmatize(r_out[i])    

    r_out = (' ').join(r_out)
    r_out = r_out.strip()
    r_out = r_out.lower()
    r_out = re.sub('\s+', ' ', r_out)
    r_out = r_out.split(' ')
    return r_out

In [ ]:
r = list(text.values)
print(len(r))
print(r[0])
print(type(r))

In [ ]:
R = []
start = time.time()
timer1 = start
len_r = len(r)
errors = []
for i,r1 in enumerate(r):
    if(i%100==0):
        timer2 = time.time()
        print(i,norm(timer2-timer1))
        timer1 = timer2
    if(i%1000==0 and i != 0):
        t_so_far = norm((timer2-start)/60,0)
        print(norm(i/len_r)*100,'% done in ',t_so_far,' minutes\t',norm((t_so_far*(len_r-i))/i,0),' minutes remaining...')
    try:
        r1 = r1[2:-1]
        new2 = pre2(r1)
        R.append(new2)
    except:
        print('\t',i,r1)
        errors.append(i)
    
timer2 = time.time()
print(i,norm(timer2-timer1))

In [ ]:
print(len(R),len(r),len(errors))

In [ ]:
#df.loc[errors[0],:]
df.drop(df.index[errors],inplace=True)
print(errors[0:10])
print(len(df))
#,len(df1))

In [ ]:
#df1.loc[errors[0],:]
df['text'] = R

In [ ]:
df.iloc[172,:]

In [ ]:
df.to_csv('cleaned.csv')

### Now cleaned. Let's pick the most frequent ones.

In [ ]:
freq = df['specialty'].value_counts().to_dict()
import operator
sorted_freq = sorted(freq.items(), key=operator.itemgetter(1),reverse=True)
sorted_freq

In [ ]:
threshold = 1000
allowed = [] # list of allowed specialties, which have frequency above threshold
for i in sorted_freq:
    if i[1] >= threshold:
        allowed.append(i[0])
    else:
        break
print(allowed)
print(len(allowed))

In [ ]:
freq = {}
ignores = []
for i,s in enumerate(df['specialty']):
    if(s in allowed):
        try:
            freq[s]+=1
            if(freq[s]>threshold):
                ignores.append(i)
        except:
            freq[s] = 1
    else:
        ignores.append(i)
df.drop(df.index[ignores],inplace=True)

In [ ]:
print(len(df),ignores[0],df.iloc[ignores[0],:])

In [ ]:
df.to_csv('subset.csv')

In [ ]:
df.to_pickle('subset.pickle',protocol=2)